# Train SqueezeNet model on tif for multi-label classification of rare tags

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [3]:
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [4]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

from data_utils import get_id_type_list_for_class, OUTPUT_PATH, GENERATED_DATA, to_set, RESOURCES_PATH
from training_utils import classification_train as train, classification_validate as validate
from training_utils import exp_decay, step_decay, get_high_zoom_imgaug_seq

Using TensorFlow backend.


In [5]:
from models.squeezenet_tif_classification import get_squeezenet_on_tif

from sklearn.model_selection import KFold
from data_utils import to_set, equalized_data_classes, unique_tags, train_jpg_ids, TRAIN_ENC_CL_CSV
from data_utils import load_pretrained_model, get_label
from data_utils import DataCache

from xy_providers import tif_image_label_provider
from models.keras_metrics import binary_crossentropy_with_false_negatives


# Setup configuration

seed = 2017
np.random.seed(seed)

cache = DataCache(6000)  # !!! CHECK BEFORE LOAD TO FLOYD

class_index = 0

trainval_id_type_list = get_id_type_list_for_class(class_index, 'Train_tif')

# ### ADD GENERATED IMAGES
# from glob import glob
# from data_utils import GENERATED_DATA, to_set, get_label

# gen_train_files = glob(os.path.join(GENERATED_DATA, "train", "tif", "*.tif"))
# gen_train_ids = [s[len(os.path.join(GENERATED_DATA, "train", "tif"))+1+len('gen_train_'):-4] for s in gen_train_files]
# gen_id_type_list = [(image_id, "Generated_Train_tif") for image_id in gen_train_ids]
# class_index_gen_train_ids = [id_type for id_type in gen_train_ids if np.sum(get_label(*gen_id_type_list[0], class_index=class_index)) > 0]
# class_index_gen_id_type_list = [(image_id, "Generated_Train_tif") for image_id in class_index_gen_train_ids]
# trainval_id_type_list = trainval_id_type_list + class_index_gen_id_type_list
# ### ADD GENERATED IMAGES


class_indices = list(equalized_data_classes.keys())
class_indices.remove(class_index)

n_other_samples = int(len(trainval_id_type_list) * 1.0 / len(class_indices) / len(equalized_data_classes[class_index]))

for index in class_indices:
    id_type_list = np.array(get_id_type_list_for_class(index, 'Train_tif'))
    id_type_list = list(to_set(id_type_list) - to_set(trainval_id_type_list))
    np.random.shuffle(id_type_list)
    trainval_id_type_list.extend(id_type_list[:n_other_samples])

print(len(trainval_id_type_list), len(to_set(trainval_id_type_list)))

1625 1625


In [6]:
trainval_id_type_list[0]

('10', 'Train_tif')

In [17]:
params = {
    'seed': seed,

    'xy_provider': tif_image_label_provider,

    'network': get_squeezenet_on_tif,
    'optimizer': 'nadam',
    'loss': binary_crossentropy_with_false_negatives, # 'binary_crossentropy',
    'nb_epochs': 25,    # !!! CHECK BEFORE LOAD TO FLOYD
    'batch_size': 32,  # !!! CHECK BEFORE LOAD TO FLOYD

    'normalize_data': True,
    'normalization': '',

    'train_seq': get_high_zoom_imgaug_seq,
    
    'image_size': (256, 256),

    'class_index': class_index,

    # Learning rate scheduler
    'lr_kwargs': {
        'lr': 0.01,
        'a': 0.95,
        'init_epoch': 0
    },
    'lr_decay_f': exp_decay,

    # Reduce learning rate on plateau
    'on_plateau': True,
    'on_plateau_kwargs': {
        'monitor': 'val_loss',
        'factor': 0.1,
        'patience': 3,
        'verbose': 1
    },

    'cache': cache,

#      'pretrained_model': 'load_best',
#     'pretrained_model': os.path.join(GENERATED_DATA, "weights", ""),

    'output_path': OUTPUT_PATH,
}

params['save_prefix_template'] = '{cnn_name}_tif_class=%i_fold={fold_index}_seed=%i' % (class_index, params['seed'])
params['input_shape'] = params['image_size'] + (7,)
params['n_classes'] = len(equalized_data_classes[class_index]) + 1

In [18]:
from data_utils import get_label
labels = np.zeros((len(trainval_id_type_list), len(equalized_data_classes[class_index])+1), dtype=np.uint8)
doubles = []
for i, it in enumerate(trainval_id_type_list):
    labels[i, :-1] = get_label(*it, class_index=class_index)
    if np.sum(labels[i, :-1]) < 1:
        labels[i, -1] = 1
    if np.sum(labels[i, :-1]) > 1:
        doubles.append(labels[i, :-1])

In [19]:
stats = np.sum(labels, axis=0)
stats, len(trainval_id_type_list)

(array([339, 332,  98, 100, 340, 209, 232], dtype=uint64), 1625)

In [21]:
# Start CV

n_folds = 5
val_fold_index = 0
val_fold_indices = []  # !!! CHECK BEFORE LOAD TO FLOYD
hists = []

kf = KFold(n_splits=n_folds)
trainval_id_type_list = np.array(trainval_id_type_list)
for train_index, test_index in kf.split(trainval_id_type_list):
    train_id_type_list, val_id_type_list = trainval_id_type_list[train_index], trainval_id_type_list[test_index]

    if len(val_fold_indices) > 0:
        if val_fold_index not in val_fold_indices:
            val_fold_index += 1
            continue

    val_fold_index += 1
    print("\n\n ---- Validation fold index: ", val_fold_index, "/", n_folds)

    print(datetime.now(), len(train_id_type_list), len(val_id_type_list))
    assert len(to_set(train_id_type_list) & to_set(val_id_type_list)) == 0, "WTF"

    weights = None if 'pretrained_model' in params else None
    cnn = params['network'](lr=params['lr_kwargs']['lr'], weights=weights, **params)
    params['save_prefix'] = params['save_prefix_template'].format(cnn_name=cnn.name, fold_index=val_fold_index - 1)
    print("\n {} - Loaded {} model ...".format(datetime.now(), cnn.name))

    if 'pretrained_model' in params:
        load_pretrained_model(cnn, **params)

    print("\n {} - Start training ...".format(datetime.now()))
    h = train(cnn, train_id_type_list, val_id_type_list, **params)
    if h is None:
        continue
    hists.append(h)



 ---- Validation fold index:  1 / 5
2017-07-18 00:53:27.442952 1300 325

 2017-07-18 00:53:29.317329 - Loaded Separable_SqueezeNet_BN_on_tif model ...

 2017-07-18 00:53:29.317421 - Start training ...

-- Training parameters: 32, 25, 1312, 352

-- Fit model

-- Fit stats of train dataset
Load existing file: /Users/vfomin/Documents/ML/Kaggle/PlanetAmazonRainForest/common/../output/generated/Separable_SqueezeNet_BN_on_tif_tif_class=0_fold=0_seed=2017_stats.npz
No need to recompute statistics
- New Keras API found -
Epoch 1/25
40/41 [============================>.] - ETA: 3s - loss: 0.6425 - precision: 0.4152 - recall: 0.0911
Epoch validation: f2 = 0.123077, mae=0.132747 

41/41 [==============================] - 209s - loss: 0.6384 - precision: 0.4213 - recall: 0.0919 - val_loss: 0.5184 - val_precision: 0.6985 - val_recall: 0.0955
Epoch 2/25
41/41 [==============================] - 182s - loss: 0.5025 - precision: 0.7232 - recall: 0.1008 - val_loss: 0.5590 - val_precision: 0.0000e+00 -

KeyboardInterrupt: 

In [26]:
from training_utils import get_gen_flow, get_val_imgaug_seq
from metrics import score

In [29]:
train_gen, train_flow = get_gen_flow(id_type_list=train_id_type_list,
                                     imgaug_seq=params['train_seq'](params['seed']),
                                     **params)
params2 = dict(params)
params2['normalization'] = 'from_save_prefix'

val_gen, val_flow = get_gen_flow(id_type_list=val_id_type_list,
                                 imgaug_seq=get_val_imgaug_seq(params['seed']),
                                 **params2)


-- Fit stats of train dataset
Load existing file: /Users/vfomin/Documents/ML/Kaggle/PlanetAmazonRainForest/common/../output/generated/Separable_SqueezeNet_BN_on_tif_tif_class=0_fold=0_seed=2017_stats.npz
No need to recompute statistics


In [30]:
for x, y in train_flow:
    break

In [34]:
y_pred = cnn.predict_on_batch(x)
print((y_pred > 0.35).astype(np.uint8) - 0.77 * y), 
print(score(y, (y_pred > 0.35).astype(np.uint8)))

[[ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.77  0.  ]
 [ 1.    0.    0.    0.    0.    0.   -0.77]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.   -0.77  0.    0.    0.    0.    0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 1.    0.    0.    0.    0.    0.   -0.77]
 [ 0.   -0.77  0.    0.    0.    0.    0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 0.   -0.77  0.    0.    0.    0.    0.  ]
 [ 1.    0.    0.    0.    0.   -0.77  0.  ]
 [ 0.    0.    0.    0.    0.   -0.77  0.  ]
 [ 0.   -0.77  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.77  0.  ]
 [ 1.    0.    0.    0.    0.    0.   -0.77]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.   -0.77  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.    0

In [35]:
for x2, y2 in val_flow:
    break

In [37]:
y_pred = cnn.predict_on_batch(x2)
print((y_pred > 0.35).astype(np.uint8) - 0.77 * y2), 
print(score(y2, (y_pred > 0.1).astype(np.uint8)))

[[ 1.    0.    0.   -0.77  0.    0.    0.  ]
 [ 0.    0.   -0.77  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 1.    0.    0.   -0.77  0.    0.    0.  ]
 [ 1.    0.    0.   -0.77  0.    0.    0.  ]
 [ 0.   -0.77  0.    0.    0.    0.    0.  ]
 [ 0.    0.   -0.77  0.    0.    0.    0.  ]
 [ 0.   -0.77  0.    0.    0.    0.    0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 1.    0.    0.    0.    0.   -0.77  0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 1.    0.    0.    0.    0.   -0.77  0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 1.    0.    0.   -0.77  0.    0.    0.  ]
 [ 0.   -0.77  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.77  0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.    0.    0.    0.   -0.77  0.    0.  ]
 [ 0.23  0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.77  0.  ]
 [ 1.    0

In [ ]:
# ### Validation all classes

n_runs = 2
n_folds = 5
run_counter = 0
cv_mean_scores = np.zeros((n_runs, n_folds))
val_fold_indices = []  # !!! CHECK BEFORE LOAD TO FLOYD

params['pretrained_model'] = 'load_best'

_trainval_id_type_list = np.array(trainval_id_type_list)

while run_counter < n_runs:
    run_counter += 1
    print("\n\n ---- New run : ", run_counter, "/", n_runs)
    val_fold_index = 0
    kf = KFold(n_splits=n_folds)
    for train_index, test_index in kf.split(_trainval_id_type_list):
        train_id_type_list, val_id_type_list = _trainval_id_type_list[train_index], _trainval_id_type_list[test_index]

        if len(val_fold_indices) > 0:
            if val_fold_index not in val_fold_indices:
                val_fold_index += 1
                continue

        val_fold_index += 1
        print("\n\n ---- Validation fold index: ", val_fold_index, "/", n_folds)

        print(len(train_id_type_list), len(val_id_type_list))
        assert len(to_set(train_id_type_list) & to_set(val_id_type_list)) == 0, "WTF"

        cnn = params['network'](input_shape=params['input_shape'], n_classes=params['n_classes'])
        params['save_prefix'] = params['save_prefix_template'].format(cnn_name=cnn.name, fold_index=val_fold_index-1)
        print("\n {} - Loaded {} model ...".format(datetime.now(), cnn.name))

        load_pretrained_model(cnn, **params)

        params['seed'] += run_counter - 1

        f2, mae = validate(cnn, val_id_type_list, verbose=0, **params)
        cv_mean_scores[run_counter-1, val_fold_index-1] = f2

        np.random.shuffle(_trainval_id_type_list)

print(cv_mean_scores)
